A faire : 
- contruire le graphe
- changer les noms des fichiers json pour qu'ils soient appelés par leur titre

In [ ]:
#pip install wikipedia-api

In [1]:
import wikipediaapi


wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='MyProjectName',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

##  Get links for each category

In [90]:
from random import shuffle

def url_categorymembers(categorymembers,list_pages=[], level=0, max_level=1, i=0):
    pages_cat = list(categorymembers.values()) #toutes les pages de la catégorie

    shuffle(pages_cat)

    for c in pages_cat:
        if i==1000: #1000 pages par catégorie
            break

        if c.title[:6]!="Portal" and c.title[:8]!="Category": # vérifie que c'est bien un article
            i+=1
            #list_urls.append(c.fullurl)
            #print(c.title)
            #list_pages.append(c)
            list_pages.append(c.title)

        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level: # descend d'un niveau
            i = url_categorymembers(c.categorymembers, list_pages, level=level + 1, max_level=max_level, i=i)[1]

    return list_pages, i

In [91]:
cat_arts = wiki_wiki.page("Category:The arts") # Arts
arts, _ = url_categorymembers(cat_arts.categorymembers)

In [64]:
cat_games = wiki_wiki.page("Category:Games") # Games
games, _= url_categorymembers(cat_games.categorymembers)

In [65]:
cat_youth = wiki_wiki.page("Category:Youth") # Kids and Teens (not exact)
youth, _ = url_categorymembers(cat_youth.categorymembers)

In [66]:
cat_reference = wiki_wiki.page("Category:Reference") # Reference
reference, _ = url_categorymembers(cat_reference.categorymembers)

In [67]:
cat_shopping = wiki_wiki.page("Category:Shopping (activity)") # Shopping
shopping, _ = url_categorymembers(cat_shopping.categorymembers)

In [68]:
cat_business = wiki_wiki.page("Category:Business") # Business
business, _ = url_categorymembers(cat_business.categorymembers)

In [69]:
cat_health = wiki_wiki.page("Category:Health") # Health
health, _ = url_categorymembers(cat_health.categorymembers)

In [70]:
cat_news = wiki_wiki.page("Category:News") # News
news, _ = url_categorymembers(cat_news.categorymembers)

In [71]:
cat_geography = wiki_wiki.page("Category:Geography") # Regional (not exact)
geography, _ = url_categorymembers(cat_geography.categorymembers)

In [72]:
cat_society = wiki_wiki.page("Category:Society") # Society
society, _ = url_categorymembers(cat_society.categorymembers)

In [73]:
cat_computers = wiki_wiki.page("Category:Computers") # Computers
computers, _ = url_categorymembers(cat_computers.categorymembers)

In [74]:
cat_home = wiki_wiki.page("Category:Home") # Home
home, _ = url_categorymembers(cat_home.categorymembers)

In [75]:
cat_recreation = wiki_wiki.page("Category:Recreation") # Recreation
recreation, _ = url_categorymembers(cat_recreation.categorymembers)

In [76]:
cat_science = wiki_wiki.page("Category:Science") # Science
science, _ = url_categorymembers(cat_science.categorymembers)

In [77]:
cat_sports = wiki_wiki.page("Category:Sports") # Sports
sports, _ = url_categorymembers(cat_sports.categorymembers)

In [78]:
cat_world = wiki_wiki.page("Category:World") # World
world, _ = url_categorymembers(cat_world.categorymembers)

In [79]:
topics = {
    "Arts": arts,
    "Games": games,
    "Youth": youth,
    "Reference": reference, 
    "Shopping": shopping, 
    "Business": business, 
    "Health": health, 
    "News": news,
    "Geography": geography,
    "Society": society,
    "Computers": computers,
    "Home": home,
    "Recreation": recreation,
    "Science": science,
    "Sports": sports,
    "World": world
}

## Compute Page Rank vectors (3.2)

In [3]:
import numpy as np

"""
# extract links from a Wikipedia page
def page_links(page):
        links = page.links
        links2 = []
        for p in links.values():
            try:
                url = p.fullurl
            except:
                url = None
            if url is not None:
                links2.append(url)
        return links2

# build the adjacency matrix
def build_adjacency_matrix(urls, pages):
    num_pages = len(urls)
    adjacency_matrix = np.zeros((num_pages, num_pages))

    for i, url in enumerate(urls):
        links = page_links(pages[i])
        for link in links:
            if link in urls:
                j = urls.index(link)
                adjacency_matrix[i][j] = 1

    return adjacency_matrix
"""

# compute PageRank scores
def compute_pagerank(M, alpha=0.25):
    # M = adjacency_matrix
    # damping_factor = 1-alpha
    N = M.shape[0]
    teleportation_matrix = np.ones((N, N)) / N

    M_prime = (1-alpha) * M + alpha * teleportation_matrix

    rank = np.ones(N) / N
    old_rank = np.zeros(N)

    epsilon = 1.0e-6
    max_iterations = 100
    iterations = 0

    while np.sum(np.abs(rank - old_rank)) > epsilon and iterations < max_iterations:
        old_rank = rank.copy()
        rank = np.dot(M_prime, rank)
        iterations += 1

    return rank

In [ ]:
# Compute PageRank scores for each topic

topic_pagerank = {}
for topic, (urls, pages) in topics.items():
    adjacency_matrix = build_adjacency_matrix(urls, pages)
    pagerank_scores = compute_pagerank(adjacency_matrix)
    topic_pagerank[topic] = pagerank_scores

# Print PageRank scores for each topic
for topic, scores in topic_pagerank.items():
    print(f"Topic: {topic}")
    for url, score in zip(topics[topic], scores):
        print(f"{url}: {score}\n")
    

# Similarity ranking (4.1)

In [13]:
def OSim(t1, t2, n=20):
    # t1, t2 listes
    return len(set(t1[:n])&set(t2[:n]))/n

In [ ]:
import itertools

def KSim(t1, t2):
    # t1, t2 listes
    U = set(t1)|set(t2)
    delta1 = U - set(t1)
    delta2 = U - set(t2)
    t1_prime = t1+list(delta1)
    t2_prime = t1+list(delta2)
    sim = 0
    for u,v in list(itertools.permutations(U, 2)):
        if np.sign(t1_prime.index(u)-t1_prime.index(v))==np.sign(t2_prime.index(u)-t2_prime.index(v)):
            sim+=1
    return sim/(len(U)*(len(U)-1))


# Query-time importance score (3.3)

## Calcul de D

A faire : changer le nom de fichiers json pour qu'ils soient enregistrés sous le nom de leur titre

In [ ]:
import json
from sklearn.feature_extraction.text import CountVectorizer


categories_text = []
for c in topics.keys():
    text=""
    for article in topics[c]:
        with open(f"{article}.json") as json_file:
            json_data = json.load(json_file)
            text+=" "+json_data.text
        categories_text.append(text)

vectorizer = CountVectorizer(stop_words='english')
D = vectorizer.fit_transform(categories_text)
terms_indexes = vectorizer.get_feature_names_out()
    

## Score

In [ ]:
def sqd(q, topic_pagerank, D, terms_indexes):
    # D = gives the total number of occurrences of each term in each document, array of shape = classes x terms
    
    p_c = 1/16 # "The quantity P(cj) is not as straightforward. We chose to make it uniform
    
    indexes = np.in1d(terms_indexes, np.array(q)).nonzero()[0]

    p_q_c = D[indexes]/D.sum(axis=1)

    probas_c_q = p_c * np.product(p_q_c)

    return np.sum(probas_c_q*topic_pagerank,axis=1)